In [ ]:
#| default_exp compat

In [ ]:
#| export
from typing import Union

import pandas as pd

In [ ]:
#| export
try:
    from polars import DataFrame as pl_DataFrame
    from polars import Series as pl_Series

    POLARS_INSTALLED = True
except ImportError:
    class pl_DataFrame:  # type: ignore
        ...

    class pl_Series:  # type: ignore
        ...

    POLARS_INSTALLED = False

try:
    import plotly  # noqa: F401

    PLOTLY_INSTALLED = True
except ImportError:
    PLOTLY_INSTALLED = False

try:
    import plotly_resampler  # noqa: F401

    PLOTLY_RESAMPLER_INSTALLED = True
except ImportError:
    PLOTLY_RESAMPLER_INSTALLED = False

DataFrame = Union[pd.DataFrame, pl_DataFrame]